# Chatbot de Suporte: Programmatic Prompts e Parsers de Saída (LangChain + Gemini)

Este notebook demonstra como construir um assistente de suporte aplicando:
- PromptTemplates e few-shot
- Conditional PromptSelector
- StructuredOutputParser (saída padronizada)
- Validação com Pydantic

Modelo usado: `gemini-2.0-flash` via Google Generative AI (LangChain).

In [ ]:
%pip install -q -r ../requirements.txt

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv("GOOGLE_API_KEY")
model_name = os.getenv("MODEL_NAME", "gemini-2.0-flash")

if not api_key:
    raise RuntimeError("Defina 'GOOGLE_API_KEY' no ambiente ou no arquivo .env.")

from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model=model_name, api_key=api_key, temperature=0.3)
print(f"Modelo carregado: {model_name}")

## PromptTemplates para injetar variáveis
Um PromptTemplate cria um texto com espaços para variáveis, que são preenchidos dinamicamente.

In [ ]:
from langchain.prompts import PromptTemplate

template = """
    Você é um assistente que responde de forma clara.
    Explique o conceito: {conceito}
    """

prompt = PromptTemplate.from_template(template)
texto_pronto = prompt.format(conceito="Aprendizado de Máquina")
print(texto_pronto)
print("---")
print(llm.invoke(texto_pronto).content)

## Few-shot prompting
Incluímos exemplos diretamente no prompt para ensinar o estilo de resposta ao modelo.

In [ ]:
template = """
    Você é um explicador de conceitos com exemplos curtos.

    Exemplo:
    Conceito: API
    Explicação: API é uma forma de dois sistemas conversarem entre si.

    Conceito: {conceito}
    Explicação:
"""

prompt = PromptTemplate.from_template(template)
texto_pronto = prompt.format(conceito="Banco de Dados")
print(llm.invoke(texto_pronto).content)

## Conditional PromptSelector
Escolhe automaticamente o melhor prompt com base na entrada (ex.: tamanho).

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda

short_prompt = PromptTemplate.from_template("Resuma em 1 frase: {texto}")
long_prompt  = PromptTemplate.from_template("Faça um resumo estruturado, com tópicos: {texto}")

def escolher_prompt(vars):
    return long_prompt if len(vars["texto"]) > 200 else short_prompt

selector = RunnableLambda(escolher_prompt)

# Cria a chain completa
chain = selector | llm

entrada_curta = "Modelos de linguagem ajudam em tarefas de processamento de texto."
entrada_longa = " ".join(["Texto muito longo"] * 25)

print("Curto:")
print(chain.invoke({"texto": entrada_curta}).content)

print("---")

print("Longo:")
print(chain.invoke({"texto": entrada_longa}).content)


## StructuredOutputParser
Força o LLM a retornar um formato específico (ex.: JSON) com campos definidos.

In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

schemas = [
    ResponseSchema(name="titulo", description="Título resumido do conteúdo"),
    ResponseSchema(name="tags", description="Lista de palavras-chave relacionadas")
]

parser = StructuredOutputParser.from_response_schemas(schemas)
format_instructions = parser.get_format_instructions()

prompt_text = f"""
Resuma o texto abaixo e extraia informações estruturadas.

Texto:
"A IA está transformando a indústria automotiva com análise de dados em tempo real."

{format_instructions}
"""

raw = llm.invoke(prompt_text).content
print("Bruto:", raw)
estruturado = parser.parse(raw)
print("Estruturado:", estruturado)

## Validação de tipos com Pydantic
Garante que a resposta possui os tipos corretos e permite integração segura.

In [ ]:
from pydantic import BaseModel, ValidationError
from typing import List

class Resumo(BaseModel):
    titulo: str
    tags: List[str]

try:
    obj = Resumo(**estruturado)
    print("Validação OK:", obj.model_dump())
except ValidationError as e:
    print("Erro de validação:", e)

# Chatbot de Suporte (com PromptSelector + Parser + Pydantic)
Integra todos os conceitos para uma função de atendimento de suporte com saída previsível.

In [ ]:
from typing import List, Optional
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

# =========================
# Modelo Pydantic para resposta
# =========================
class SuporteResposta(BaseModel):
    tipo: str = Field(description="Tipo de solicitação: 'faq' ou 'troubleshooting'")
    resposta: str
    passos: List[str]
    prioridade: Optional[str] = Field(default=None, description="baixa|media|alta")
    tags: List[str]

# =========================
# Parser LangChain baseado em schemas
# =========================
schemas = [
    ResponseSchema(name="tipo", description="faq ou troubleshooting"),
    ResponseSchema(name="resposta", description="Resposta clara e objetiva"),
    ResponseSchema(name="passos", description="Lista de passos recomendados"),
    ResponseSchema(name="prioridade", description="baixa|media|alta"),
    ResponseSchema(name="tags", description="Lista de palavras-chave")
]

suporte_parser = StructuredOutputParser.from_response_schemas(schemas)
suporte_format_instructions = suporte_parser.get_format_instructions()

# =========================
# Templates de prompt
# =========================
short_tpl = PromptTemplate.from_template("""
Você é um assistente de suporte conciso. Responda diretamente e estruture a saída.

Pergunta: {pergunta}

{format_instructions}
""")

long_tpl = PromptTemplate.from_template("""
Você é um assistente de suporte detalhado e empático. Identifique sintomas, causas prováveis e passos de resolução.

Pergunta: {pergunta}
Contexto: {contexto}

{format_instructions}
""")

# =========================
# RunnableLambda para seleção do prompt
# =========================
def escolher_prompt(vars):
    pergunta = vars["pergunta"].lower()
    # Se pergunta longa ou contém palavras-chave de erro -> prompt detalhado
    if len(pergunta) > 180 or any(
        k in pergunta for k in ["erro", "falha", "não funciona", "exception", "traceback"]
    ):
        return long_tpl.format(
            pergunta=vars["pergunta"],
            contexto=vars["contexto"],
            format_instructions=vars["format_instructions"]
        )
    # Caso contrário -> prompt curto
    return short_tpl.format(
        pergunta=vars["pergunta"],
        contexto=vars.get("contexto", ""),
        format_instructions=vars["format_instructions"]
    )

selector = RunnableLambda(escolher_prompt)

# =========================
# Função de atendimento com output parser
# =========================
def atender_usuario(pergunta: str, contexto: str = "") -> SuporteResposta:
    vars = {
        "pergunta": pergunta,
        "contexto": contexto,
        "format_instructions": suporte_format_instructions
    }

    # Escolhe o prompt dinamicamente e já retorna string pronta
    final_prompt = selector.invoke(vars)

    # Chama o LLM
    resposta_raw = llm.invoke(final_prompt).content

    # Parser estruturado do LangChain
    parsed = suporte_parser.parse(resposta_raw)

    # =============================
    # Garantir que 'passos' e 'tags' sejam listas
    # =============================
    if isinstance(parsed.get("passos"), str):
        # Quebra por linha ou por numeração típica "1.", "2.", etc.
        parsed["passos"] = [p.strip() for p in parsed["passos"].replace("1.", "\n1.").split("\n") if p.strip()]
    
    if isinstance(parsed.get("tags"), str):
        parsed["tags"] = [t.strip() for t in parsed["tags"].split(",") if t.strip()]

    # Retorna objeto Pydantic validado
    return SuporteResposta(**parsed)


# =========================
# Exemplo de uso
# =========================
print("Assistente preparado.")

# Testes:
# resp1 = atender_usuario("Como altero o e-mail da minha conta?")
# print(resp1.model_dump())

# resp2 = atender_usuario("Recebo erro 500 ao fazer login com FastAPI e Postgres. O que devo fazer?", contexto="Servidor de produção")
# print(resp2.model_dump())



In [ ]:
# =========================
# Teste 1: pergunta curta (FAQ)
resp1 = atender_usuario("Como altero o e-mail da minha conta?")
print("=== Pergunta Curta ===")
print(resp1.model_dump())

# =========================
# Teste 2: troubleshooting
resp2 = atender_usuario(
    "Recebo erro 500 ao salvar pedido no checkout. Uso Node, Express e MongoDB.",
    contexto="Versão 3.2 do checkout, deploy no Vercel."
)
print("=== Pergunta Longa / Troubleshooting ===")
print(resp2.model_dump())
